<a href="https://colab.research.google.com/github/tantowijh/colab-collection/blob/main/WebServer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive logo"/></center>
#@markdown <center><h3>Mount Gdrive to /content/drive</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")
get_ipython().system_raw('find . -type d -name ".ipynb_checkpoints" -prune -exec rm -rf {} \; &')

Mounted at /content/drive


In [ ]:
#@title <h4><b>Web Server Configuration</b></h4>

### Install ngrok
from time import sleep
from IPython.utils import io
with io.capture_output() as captured:
  !curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok

### Run ngrok to tunnel Dash app port 8050 to the outside world. 
### This command runs in the background.
NgrokToken = "2DW9ORIBnygvyr8biHB0pGhEA0v_7hxAdbpPCaRMd6gu48C6E" #@param {type:"string"}
NgrokRegion = "us" #@param ["us", "eu", "ap", "au", "sa", "jp", "in"]
!ngrok config add-authtoken {NgrokToken}
get_ipython().system_raw('ngrok http 8050 --region {NgrokRegion} &')


def serverstart():
  #@title <h4><b>Start Web Server</b></h4>

  ### Get the public URL.
  !curl -s http://localhost:4040/api/tunnels | python3 -c \
      "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

  #directory for server path
  ServerLocationPath = "/content/drive/Shareddrives/01. Anime Server" #@param {type:"string"}
  from IPython.utils import io
  with io.capture_output() as captured:
    %cd '{ServerLocationPath}'
  !python -m http.server 8050

sleep(3); serverstart()
